<a href="https://colab.research.google.com/github/Meph1sto14/Project_Capstone_IBM_Skillsbuild_Data/blob/main/Project_Capstone_IBM_Skillsbuild_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q replicate pandas matplotlib seaborn
!pip install -q -U langchain langchain_community langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from google.colab import userdata
from langchain_community.llms import Replicate
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent


In [3]:
api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [4]:
zegion_raw = Replicate(model="ibm-granite/granite-3.3-8b-instruct")

In [7]:
df = pd.read_csv("renewable_energy_dataset.csv")
print("🔹 Dataset Shape:", df.shape)
print("🔹 Columns:", df.columns.tolist())
display(df.head())

🔹 Dataset Shape: (500, 19)
🔹 Columns: ['Timestamp', 'Solar_Irradiance', 'Temperature', 'Wind_Speed', 'Humidity', 'Rainfall', 'Solar_Power_Generated', 'Wind_Power_Generated', 'Hydro_Power_Generated', 'Battery_Level', 'Battery_Input_Output', 'Aux_Backup_Power', 'Load_Demand', 'Grid_Frequency', 'Energy_Cost', 'Forecasted_Solar', 'Forecasted_Wind', 'Forecasted_Load', 'Optimal_Distribution']


,Timestamp,Solar_Irradiance,Temperature,Wind_Speed,Humidity,Rainfall,Solar_Power_Generated,Wind_Power_Generated,Hydro_Power_Generated,Battery_Level,Battery_Input_Output,Aux_Backup_Power,Load_Demand,Grid_Frequency,Energy_Cost,Forecasted_Solar,Forecasted_Wind,Forecasted_Load,Optimal_Distribution
0,2025-01-01 00:00:00,599.342831,29.630888,8.798711,46.189243,0.243483,179.802849,52.429184,228.224397,52.865639,33.527861,91.783240,354.947493,50.048618,0.191591,168.518926,51.512721,355.431237,477.779191
1,2025-01-01 01:00:00,472.347140,34.547083,7.849267,79.502965,0.208051,141.704142,37.222310,276.321462,43.936307,-18.910138,110.604668,467.874688,49.805063,0.145410,111.823386,25.990313,472.173877,448.559738
2,2025-01-01 02:00:00,629.537708,18.007162,6.119261,59.895341,0.389797,188.861312,17.636517,244.216407,42.138158,-23.067010,50.332065,352.248758,49.794964,0.159376,213.964228,23.888541,336.771599,314.315675
3,2025-01-01 03:00:00,804.605971,27.814846,4.706126,34.623497,0.083704,241.381791,8.022445,127.357096,43.643695,22.746291,49.206059,297.508987,50.073918,0.188393,231.428341,25.529257,269.747219,495.747976
4,2025-01-01 04:00:00,453.169325,21.746787,7.396447,33.513056,0.403395,135.950798,31.144785,264.699226,44.562868,6.771058,30.572872,389.434950,50.050702,0.091826,110.588061,30.799027,336.771813,418.293697


In [8]:
agent = create_pandas_dataframe_agent(
    zegion_raw,
    df=df,
    verbose=False,
    allow_dangerous_code=True,
)

def safe_query(prompt):
    try:
        return agent.invoke(prompt)
    except Exception as e:
        return f"⚠️ Error: {e}"

In [9]:
queries = [
    "Berapa rata-rata Solar_Power_Generated per jam?",
    "Bandingkan Forecasted_Load dengan Load_Demand.",
    "Jam berapa Wind_Power_Generated paling tinggi?",
]

for q in queries:
    print(f"\n❓ {q}")
    print("🤖", safe_query(q))


❓ Berapa rata-rata Solar_Power_Generated per jam?
🤖 {'input': 'Berapa rata-rata Solar_Power_Generated per jam?', 'output': 'The average `Solar_Power_Generated` per hour is approximately 150.26 units.'}

❓ Bandingkan Forecasted_Load dengan Load_Demand.
🤖 {'input': 'Bandingkan Forecasted_Load dengan Load_Demand.', 'output': "Here's the comparison of 'Forecasted_Load' and 'Load_Demand' for each timestamp in the dataframe `df`:\n\n| Timestamp           | Load_Demand | Forecasted_Load | Load_Demand_Diff |\n|---------------------|-------------|-----------------|-----------------|\n| 2025-01-01 00:00:00 |   354.947493 |       355.431237 |          0.483744 |\n| 2025-01-01 01:00:00 |   467.874688 |       472.173877 |          4.299189 |\n| 2025-01-01 02:00:00 |   352.248758 |       336.771599 |        -15.477159 |\n| 2025-01-01 03:00:00 |   297.508987 |       269.747219 |        -27.761768 |\n| 2025-01-01 04:00:00 |   389.434950 |       336.771813 |        -52.663137 |\n| ...                 

In [62]:
def plot_interactive(df, x_col, y_cols, title="Grafik Interaktif"):
    """
    Membuat grafik garis interaktif yang bersih, label nilai hanya muncul saat hover.
    """
    if not isinstance(y_cols, list):
        y_cols = [y_cols]

    fig = px.line(df, x=x_col, y=y_cols, title=title)

    # UBAH DI SINI: Ganti mode menjadi 'lines+markers' saja
    fig.update_traces(
        mode="lines+markers"
    )

    fig.update_layout(
        plot_bgcolor="white",
        paper_bgcolor="white",
        font=dict(size=14),
        legend_title_text='Legenda'
    )

    fig.show()

def plot_interactive_bar(df, x_col, y_col, title="Diagram Batang Interaktif"):
    # Menggunakan y_col langsung karena kita memplot satu set nilai
    fig = px.bar(
        df,
        x=x_col,
        y=y_col,
        title=title,
        color=y_col,
        # 💡 PENTING: Lewatkan kolom nilai ke parameter 'text'
        text=y_col
    )

    # Gunakan texttemplate untuk format angka, bukan textinfo
    fig.update_traces(
        # Format teks di sini
        texttemplate='%{text:,.2f}',
        textposition='outside'
    )

    fig.update_layout(
        xaxis_title=x_col,
        yaxis_title=y_col,
        plot_bgcolor="white",
        paper_bgcolor="white",
        bargap=0.3,
        font=dict(size=14)
    )
    fig.show()

def plot_interactive_pie(df, names_col, values_col, title="Diagram Lingkaran Interaktif"):
    """Membuat diagram lingkaran interaktif."""
    fig = px.pie(df, names=names_col, values=values_col, title=title)

    fig.update_traces(textposition='inside', textinfo='percent+label')

    fig.show()

In [63]:
def smart_invoke(prompt, df=None, zegion=None):
    p_lower = prompt.lower()

    # 1. Ekstrak nama kolom yang valid dari prompt secara otomatis
    mentioned_cols = [col for col in df.columns if col.lower() in p_lower]

    # 2. Logika untuk memilih jenis visualisasi

    # >> Logika Diagram Batang (Sudah Diperbaiki)
    if "diagram batang" in p_lower:
        # Memastikan ada kolom yang disebut dalam prompt
        if not mentioned_cols:
            return "⚠️ Mohon sebutkan nama kolom yang valid untuk dibuatkan diagram batang."

        # Cek apakah user meminta 'rata-rata'
        if "rata-rata" in p_lower or "average" in p_lower:
            # Hitung rata-rata HANYA untuk kolom numerik yang disebut
            avg_data = {
                col: df[col].mean()
                for col in mentioned_cols
                if pd.api.types.is_numeric_dtype(df[col])
            }

            # Jika tidak ada kolom numerik yang valid
            if not avg_data:
                return f"⚠️ Kolom yang Anda sebutkan ({', '.join(mentioned_cols)}) tidak dapat dihitung rata-ratanya."

            # Siapkan DataFrame baru yang berisi hasil rata-rata
            plot_df = pd.DataFrame(list(avg_data.items()), columns=['Kategori', 'Nilai Rata-rata'])

            # Panggil fungsi plot_interactive_bar dengan data yang sudah diproses
            plot_interactive_bar(
                plot_df,
                x_col='Kategori',
                y_col='Nilai Rata-rata',
                title=f"Rata-rata untuk {', '.join(avg_data.keys())}"
            )
            return "✅ Diagram batang rata-rata berhasil ditampilkan."
        else:
            # Fallback jika meminta diagram batang tapi bukan rata-rata
            # (bisa dikembangkan untuk 'total', 'maksimum', dll.)
            return "⚠️ Saat ini hanya mendukung 'diagram batang rata-rata'. Mohon spesifikkan permintaan Anda (contoh: 'buat diagram batang rata-rata ...')."

    # >> Logika untuk plot lainnya (tidak berubah)
    elif "diagram lingkaran" in p_lower or "pie chart" in p_lower:
        if not mentioned_cols:
            mentioned_cols = ['Solar_Power_Generated', 'Wind_Power_Generated', 'Hydro_Power_Generated']
        total_data = {col: df[col].sum() for col in mentioned_cols if pd.api.types.is_numeric_dtype(df[col])}
        pie_df = pd.DataFrame(list(total_data.items()), columns=['Sumber', 'Total Produksi'])
        plot_interactive_pie(pie_df, names_col='Sumber', values_col='Total Produksi', title=f"Proporsi Total Produksi Energi")
        return "✅ Diagram lingkaran ditampilkan."

    elif "grafik" in p_lower or "tren" in p_lower:
        if not mentioned_cols:
            return "⚠️ Mohon sebutkan kolom yang ingin dibuatkan grafik."

        # Salin DataFrame agar tidak mengubah data asli
        plot_df = df.copy()

        # Pastikan 'Timestamp' adalah tipe datetime
        plot_df['Timestamp'] = pd.to_datetime(plot_df['Timestamp'])
        plot_df.set_index('Timestamp', inplace=True)

        # Cek apakah user meminta agregasi (rata-rata, total, dll.)
        # 'D' untuk Harian (Daily), 'W' untuk Mingguan (Weekly), 'M' untuk Bulanan (Monthly)
        resample_rule = None
        agg_func = None
        title_prefix = ""

        if "harian" in p_lower or "hari" in p_lower:
            resample_rule = 'D'
        elif "mingguan" in p_lower or "minggu" in p_lower:
            resample_rule = 'W'

        if "rata-rata" in p_lower:
            agg_func = 'mean'
            title_prefix = "Rata-Rata Harian"
        elif "total" in p_lower or "jumlah" in p_lower:
            agg_func = 'sum'
            title_prefix = "Total Harian"

        # Jika ada permintaan agregasi, proses datanya
        if resample_rule and agg_func:
            # Lakukan resampling dan agregasi
            plot_df = plot_df[mentioned_cols].resample(resample_rule).agg(agg_func)
            # Reset index agar 'Timestamp' kembali menjadi kolom
            plot_df.reset_index(inplace=True)

            # Buat judul yang dinamis
            title = f"Grafik {title_prefix} untuk {', '.join(mentioned_cols)}"
        else:
            # Jika tidak ada agregasi, gunakan data mentah
            title = f"Grafik Tren untuk {', '.join(mentioned_cols)}"
            plot_df.reset_index(inplace=True) # Reset index untuk data mentah juga

        # Panggil fungsi plot dengan DataFrame yang sudah diproses
        plot_interactive(
            plot_df,
            x_col="Timestamp",
            y_cols=mentioned_cols,
            title=title
        )
        return "✅ Grafik interaktif ditampilkan."

    elif "tabel" in p_lower:
        display(df.head())
        return "✅ Tabel ditampilkan."


    # 3. Fallback jika tidak ada kata kunci visualisasi
    # Meneruskan prompt ke agent LLM untuk dijawab
    if zegion:
        # Menggunakan agent yang sudah dibuat sebelumnya
        return agent.invoke(prompt)
    else:
        return "⚠️ LLM tidak tersedia"

In [64]:
# === Wrapper Class ===
class SmartZegion:
    def __init__(self, zegion, df):
        self.zegion = zegion
        self.df = df

    def invoke(self, prompt):
        return smart_invoke(prompt, df=self.df, zegion=self.zegion)

# === Pakai Zegion pintar ===
zegion = SmartZegion(zegion_raw, df)

In [67]:
print("🤖", zegion.invoke("Gambarkan grafik garis perkembangan Wind_Power_Generated."))

🤖 ✅ Grafik interaktif ditampilkan.
